In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# 対象者抽出


In [ ]:

# ===== 設定値 =====
BASE_DIR = Path(r"data\raw_excel")   # フォルダパスに書き換え

EXCEL_PATTERN = "★機器遠隔操作履歴＆機器状態変化履歴_*.xlsx"
TARGET_CSV = "対象者リスト.csv"

REMOTE_SHEET = "機器遠隔操作履歴"
STATE_SHEET  = "機器状態変化履歴"

# ここに「欲しいカラム名」を列挙してください
REMOTE_KEEP_COLS = [
    "ContractId",
    "OrderReceiptDate", # -> タイムスタンプ
    "TimerDiv",         # -> 操作区分 (タイマー種別)"
    "FloorCode",
    "RoomName",
    "EquipmentTypeId",
    "EquipmentName",
    "PropertyCode",     # -> 実行コード
    "PropertyName",     # -> 実行コードの日本語説明
    "PropertyValue",
]
STATE_KEEP_COLS = [
    "MessageName",      # -> 通知種別 (生きているか)
    "ContractId",       
    "ReportedDate",     # -> タイムスタンプ
    "FloorCode",
    "RoomName",
    "EquipmentTypeId",
    "EquipmentName",
    "PropertyCode1",
    "PropertyName1",
    "PropertyValue1",
]

# 出力ファイル名
REMOTE_OUT = BASE_DIR / "機器遠隔操作履歴_抽出結果.csv"
STATE_OUT  = BASE_DIR / "機器状態変化履歴_抽出結果.csv"
# ===================

In [ ]:
# --- 対象者リストの ContractId を読み込み ---
target_df = pd.read_csv(BASE_DIR / TARGET_CSV, dtype={"ContractId": str})
target_ids = set(target_df["ContractId"].astype(str))

# 出力ファイルが既にあれば削除（毎回作り直したい場合）
for out_path in [REMOTE_OUT, STATE_OUT]:
    if out_path.exists():
        out_path.unlink()

# --- Excel ファイルを1つずつ処理 ---
for xlsx_path in BASE_DIR.glob(EXCEL_PATTERN):
    print(f"processing: {xlsx_path.name}")

    # ▼ 機器遠隔操作履歴シート
    remote_df = pd.read_excel(
        xlsx_path,
        sheet_name=REMOTE_SHEET,
        dtype={"ContractId": str},  # 型を揃える
        engine="openpyxl"
    )
    # ContractId でフィルタ
    remote_df = remote_df[remote_df["ContractId"].astype(str).isin(target_ids)]

    # 指定カラムだけ残す（存在チェック付き）
    remote_cols = [c for c in REMOTE_KEEP_COLS if c in remote_df.columns]
    remote_df = remote_df[remote_cols]

    # CSV に追記
    remote_df.to_csv(
        REMOTE_OUT,
        mode="a",
        index=False,
        encoding="utf-8-sig",
        header=not REMOTE_OUT.exists()  # 最初の1回だけヘッダを書く
    )

    # ▼ 機器状態変化履歴シート
    state_df = pd.read_excel(
        xlsx_path,
        sheet_name=STATE_SHEET,
        dtype={"ContractId": str},
        engine="openpyxl"
    )
    state_df = state_df[state_df["ContractId"].astype(str).isin(target_ids)]
    state_cols = [c for c in STATE_KEEP_COLS if c in state_df.columns]
    state_df = state_df[state_cols]

    state_df.to_csv(
        STATE_OUT,
        mode="a",
        index=False,
        encoding="utf-8-sig",
        header=not STATE_OUT.exists()
    )

    # ここで xlsx_path はスコープを抜けるので、
    # 次のループに行く頃には不要になりメモリも解放されていきます。